# ASSIGNMENT: Cartoon Face Mask
## This assignment tests your ability to apply face recognition, image masks, image thresholding, video capturing and saving video feed into a video file concepts.

## TASK: Cartoonify faces in video feed from live webcam

### Steps
- 1. **Capture video** feed from webcam
- 2. **Recognize faces** in the video
- 3. **Replace/Mask the face** region with your favorite cartoon character
- 4. **Save the video** feed into a video file
- 5. Submit the following files
    - Notebook (.ipynb)
    - Video (.avi/.mp4)

In [1]:

import cv2
import os

# Set up paths for Haar Cascade XML files
cascPathface = "/Users/bishakha/Downloads/haarcascade_frontalface_alt2.xml"
cascPatheyes = "/Users/bishakha/Downloads/haarcascade_eye_tree_eyeglasses.xml"

# Load Haar Cascade classifiers
faceCascade = cv2.CascadeClassifier(cascPathface)
eyeCascade = cv2.CascadeClassifier(cascPatheyes)

In [3]:
import numpy as np

In [5]:

video_capture = cv2.VideoCapture(0)
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video_capture.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 60
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video_masked.mp4', fourcc, fps, (frame_width, frame_height))       

In [7]:
# Load your cartoon image with transparency preserved
cartoon = cv2.imread("cute.png", cv2.IMREAD_UNCHANGED)
if cartoon is None:
    raise FileNotFoundError("Could not find 'cute.png'. Ensure it is in the working directory.")

# Scale factor to increase the size of the cartoon overlay relative to the detected face
scale_factor = 1.26

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)

    for (x, y, w, h) in faces:
        # Calculate new dimensions for the cartoon overlay
        new_w = int(w * scale_factor)
        new_h = int(h * scale_factor)
         # Shift the overlay so it remains centered on the detected face
        x_offset = x - (new_w - w) // 2
        y_offset = y - (new_h - h) // 2
        
        # Make sure the new region falls within the frame boundaries
        x_offset = max(0, x_offset)
        y_offset = max(0, y_offset)
        if x_offset + new_w > frame_width:
            new_w = frame_width - x_offset
        if y_offset + new_h > frame_height:
            new_h = frame_height - y_offset
        
        # Resize the cartoon image to the new dimensions
        resized_cartoon = cv2.resize(cartoon, (new_w, new_h), interpolation=cv2.INTER_AREA)
        
        # Alpha blending if the resized cartoon has an alpha channel
        if resized_cartoon.shape[2] == 4:
            cartoon_color = resized_cartoon[:, :, :3].astype(float)
            alpha_channel = resized_cartoon[:, :, 3].astype(float) / 255.0
            alpha_mask = cv2.merge([alpha_channel, alpha_channel, alpha_channel])
            
            # Get the region of interest (ROI) from the frame
            roi = frame[y_offset:y_offset+new_h, x_offset:x_offset+new_w].astype(float)
            
            # Blend the cartoon overlay with the ROI using per-pixel alpha blending
            blended = (alpha_mask * cartoon_color) + ((1 - alpha_mask) * roi)
            blended = np.clip(blended, 0, 255).astype(np.uint8)
            frame[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = blended
        else:
            # If no alpha channel, simply replace the face region with the resized cartoon
            frame[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized_cartoon

    m_frame = cv2.flip(frame, 1)
      # Mirror (flip horizontally) the processed frame before displaying and saving
    out.write(m_frame)
    cv2.imshow('Cartoon masked Video ', m_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

2025-03-28 02:31:47.032 python[37462:3180070] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-28 02:31:47.032 python[37462:3180070] +[IMKInputSession subclass]: chose IMKInputSession_Modern


# Release Resources and Close Windows

In [10]:
# Release the video capture and writer objects
video_capture.release()
out.release()

# Close all OpenCV windows
cv2.destroyAllWindows()
